In [12]:
import pyspark
import os
import sys
from pyspark import SparkContext
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import *

In [14]:
spark = SparkSession.builder.config("spark.driver.memory", "8g").appName('chaptershhqac').getOrCreate()

ConnectionRefusedError: [Errno 111] Connection refused

In [3]:
df=spark.read.csv("/home/lplab/Desktop/ratings.csv",inferSchema=True,header=True)
df.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|      1|   4.0|1225734739|
|     1|    110|   4.0|1225865086|
|     1|    158|   4.0|1225733503|
|     1|    260|   4.5|1225735204|
|     1|    356|   5.0|1225735119|
|     1|    381|   3.5|1225734105|
|     1|    596|   4.0|1225733524|
|     1|   1036|   5.0|1225735626|
|     1|   1049|   3.0|1225734079|
|     1|   1066|   4.0|1225736961|
+------+-------+------+----------+
only showing top 10 rows



In [4]:
# Convert Unix epoch timestamp to datetime type
df = df.withColumn("datetime", from_unixtime(col("timestamp").cast("bigint")))

# Extract year from the datetime
df = df.withColumn("year", year(col("datetime")))
df.show(20)

+------+-------+------+----------+-------------------+----+
|userId|movieId|rating| timestamp|           datetime|year|
+------+-------+------+----------+-------------------+----+
|     1|      1|   4.0|1225734739|2008-11-03 23:22:19|2008|
|     1|    110|   4.0|1225865086|2008-11-05 11:34:46|2008|
|     1|    158|   4.0|1225733503|2008-11-03 23:01:43|2008|
|     1|    260|   4.5|1225735204|2008-11-03 23:30:04|2008|
|     1|    356|   5.0|1225735119|2008-11-03 23:28:39|2008|
|     1|    381|   3.5|1225734105|2008-11-03 23:11:45|2008|
|     1|    596|   4.0|1225733524|2008-11-03 23:02:04|2008|
|     1|   1036|   5.0|1225735626|2008-11-03 23:37:06|2008|
|     1|   1049|   3.0|1225734079|2008-11-03 23:11:19|2008|
|     1|   1066|   4.0|1225736961|2008-11-03 23:59:21|2008|
|     1|   1196|   3.5|1225735441|2008-11-03 23:34:01|2008|
|     1|   1200|   3.5|1225735861|2008-11-03 23:41:01|2008|
|     1|   1210|   4.5|1225735210|2008-11-03 23:30:10|2008|
|     1|   1214|   4.0|1225736426|2008-1

In [5]:
df=df.drop("timestamp")
df.show()

+------+-------+------+-------------------+----+
|userId|movieId|rating|           datetime|year|
+------+-------+------+-------------------+----+
|     1|      1|   4.0|2008-11-03 23:22:19|2008|
|     1|    110|   4.0|2008-11-05 11:34:46|2008|
|     1|    158|   4.0|2008-11-03 23:01:43|2008|
|     1|    260|   4.5|2008-11-03 23:30:04|2008|
|     1|    356|   5.0|2008-11-03 23:28:39|2008|
|     1|    381|   3.5|2008-11-03 23:11:45|2008|
|     1|    596|   4.0|2008-11-03 23:02:04|2008|
|     1|   1036|   5.0|2008-11-03 23:37:06|2008|
|     1|   1049|   3.0|2008-11-03 23:11:19|2008|
|     1|   1066|   4.0|2008-11-03 23:59:21|2008|
|     1|   1196|   3.5|2008-11-03 23:34:01|2008|
|     1|   1200|   3.5|2008-11-03 23:41:01|2008|
|     1|   1210|   4.5|2008-11-03 23:30:10|2008|
|     1|   1214|   4.0|2008-11-03 23:50:26|2008|
|     1|   1291|   5.0|2008-11-03 23:23:29|2008|
|     1|   1293|   2.0|2008-11-03 23:07:22|2008|
|     1|   1376|   3.0|2008-11-03 23:02:19|2008|
|     1|   1396|   3

In [6]:

als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating",
          coldStartStrategy="drop")

# Split data into training and test sets
(training, test) = df.randomSplit([0.8, 0.2])

# Fit ALS model on the training data
model = als.fit(training)

# Generate predictions on the test data
predictions = model.transform(test)

# Show sample predictions
predictions.show(5)


[15.989s][warning][gc,alloc] Executor task launch worker for task 2.0 in stage 6.0 (TID 31): Retried waiting for GCLocker too often allocating 131074 words
[16.004s][warning][gc,alloc] Executor task launch worker for task 19.0 in stage 6.0 (TID 48): Retried waiting for GCLocker too often allocating 131074 words
[16.117s][warning][gc,alloc] Executor task launch worker for task 18.0 in stage 6.0 (TID 47): Retried waiting for GCLocker too often allocating 131074 words


24/04/12 14:57:51 ERROR Executor: Exception in task 2.0 in stage 6.0 (TID 31)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:64)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:363)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:106)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:555)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)


[16.204s][warning][gc,alloc] Executor task launch worker for task 4.0 in stage 6.0 (TID 33): Retried waiting for GCLocker too often allocating 1048578 words
[16.218s][warning][gc,alloc] Executor task launch worker for task 4.0 in stage 6.0 (TID 33): Retried waiting for GCLocker too often allocating 1048578 words


24/04/12 14:57:51 WARN TaskSetManager: Lost task 12.0 in stage 6.0 (TID 41) (lpcp-17 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 19 in stage 6.0 failed 1 times, most recent failure: Lost task 19.0 in stage 6.0 (TID 48) (lpcp-17 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.nio.HeapByteBuffer.<init>(HeapByteBuffer.java:64)
	at java.base/java.nio.ByteBuffer.allocate(ByteBuffer.java:363)
	at org.apache.spark.io.ReadAheadInputStream.<init>(ReadAheadInputStream.java:106)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillReader.<init>(UnsafeSorterSpillReader.java:74)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.getReader(UnsafeSorterSpillWriter.java:159)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.getSortedIterator(UnsafeExternalSorter.java:555)
	at org.apache.spark.sql.execution.UnsafeExternalRowSorter.sort(UnsafeExternalRowSorter.java:172)
	at o

Py4JError: py4j does not exist in the JVM